## Apply ipyvolume to display 3D dust map in virtual reality
#### centered on the Aquila South cloud, cut out distance < 1kpc data, voxel value is magnitudes of AV converted from Bayestar reddenings,  as following the similar procedures of [plotting_the_dust_map](http://dustmaps.readthedocs.io/en/latest/examples.html#plotting-the-dust-maps)
- ipyvolume documentation is on http://ipyvolume.readthedocs.io/en/latest/vr.html
- get 3D dust map data on http://dustmaps.readthedocs.io/en/latest/examples.html#getting-started


In [76]:
from dustmaps.config import config
config['data_dir'] = '/Users/penny/Works/bayestar_dustmap'

import dustmaps.bayestar
dustmaps.bayestar.fetch()


File exists. Not overwriting.


In [68]:
from __future__ import print_function
from astropy.coordinates import SkyCoord
from dustmaps.bayestar import BayestarQuery
import numpy as np
import astropy.units as units

def get_av_cube():
    # coords = SkyCoord(180., 0., unit='deg', frame='galactic')
    bayestar = BayestarQuery(max_samples=1)

    # set up a grid of coordinates to plot, centered on the Aquila South cloud
    l0, b0 = (37., -16.)
    l = np.arange(l0 - 5., l0 + 5., 0.05)
    b = np.arange(b0 - 5., b0 + 5., 0.05)
    l, b = np.meshgrid(l, b)
    coords = SkyCoord(l*units.deg, b*units.deg, frame='galactic')
    
    # TODO: why in matplotlib the value is squared?
#     np.sqrt(Av)[::,::-1]

    # used the coefficient from Table 6 of Schlafly & Finkbeiner (2011) to convert SFD and Bayestar reddenings to magnitudes of AV.
    av_cube = 2.742*bayestar(coords, mode='median')
    
    print(av_cube.shape)
    return av_cube

av_cube = get_av_cube()


(200, 200, 31)


In [69]:
# sample cube using distances as z index

def resample_cube(cube):
    distances = np.array(bayestar.distances)
    unit = bayestar.distances.unit
    d_shape = int(distances[-1])
    sam_ebv_cube = np.zeros((cube.shape[0], cube.shape[1], d_shape+1))
    print(sam_ebv_cube.shape, cube.shape[2])
    for item in range(cube.shape[2]):
        sam_ebv_cube[:, :, int(distances[item])] += cube[:, :, item]

    # cut distance < 1kpc part
    sam_ebv_cube[:, :, 0] = 0
    print(sam_ebv_cube.shape, np.nanmax(sam_ebv_cube), np.nanmin(sam_ebv_cube))
    print(np.nanmax(sam_ebv_cube[:, :, 63]))
    return sam_ebv_cube
print(av_cube.shape)
av_cube = resample_cube(av_cube)


(200, 200, 31)


In [71]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import ipyvolume
from astropy.io import fits
# co_data = fits.open('l1448_13co.fits')

# aqa2 = ipyvolume.datasets.aquariusA2.fetch()
# level – level(s) for the where the opacity in the volume peaks, maximum sequence of length 3
ipyvolume.quickvolshow(av_cube.T, stero=True, data_min=None, level=[0.22, 0.25, 0.2], lighting=True,  width=512, height=256, stereo=True, opacity=0.06)

In [75]:
# If wee extract slice[i] from slice[i-1] for the av_cube

def sort_cube(np_array):
    d = np_array.shape[2]
    sort_array = np.zeros(np_array.shape)
    for i in list(reversed(range(d))):
        if d>0:
            sort_array[:, :, i] = np_array[:, :, i] - np_array[:, :, i-1]
    print(sort_array.shape)
    return sort_array
av_cube = get_av_cube()
sort_av_cube = sort_cube(av_cube)
print(av_cube.shape)
sort_av_cube = resample_cube(sort_av_cube)

ipyvolume.quickvolshow(sort_av_cube.T, stero=True, data_min=None, level=[0.0, 0.0, 0.0], lighting=True,  width=512, height=256, stereo=True, opacity=0.06)

(200, 200, 31)
(200, 200, 31)
(200, 200, 31)
